
# Objetivo 2: Generación de modelos

Pipeline completo para la predicción del crecimiento de datos de biodiversidad en GBIF.

Este script implementa el flujo de trabajo de principio a fin para modelar datos de panel
de series temporales, incluyendo:
1.  Carga datos de PA_dataAnalysis y preparación
2.  Ingeniería de características temporales (lags y ventanas móviles).
3.  Un marco de validación cruzada robusto para series de tiempo (ventana expansiva).
4.  Preprocesamiento (imputación y escalado) dentro del bucle de validación para evitar fuga de datos.
5.  Entrenamiento y evaluación comparativa de cuatro modelos:
    - Prophet.
    - Random Forest.
    - XGBoost.
    - Red Neuronal LSTM (para modelado secuencial).
6.  Selección del mejor modelo basado en métricas de rendimiento (MAE, RMSE, R²).
7.  Reentrenamiento del modelo final y generación de pronósticos para Colombia hasta 2030
    bajo dos escenarios de políticas.

In [38]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

# Preprocesamiento y modelado de Scikit-Learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modelos especializados

import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

# Configuraciones generales
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)

In [41]:
# Modelado de Deep Learning con TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#Carga de datos

url = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/Data_final.csv"
Data_final = pd.read_csv(url)

# Eliminación de columnas por tener muchos vacíos y no ser posible completarlas con imputación. (pensar en otras estrategias)
columns_to_drop = ['Overall score', 'areas_protegidas']
Data_final = Data_final.drop(columns=columns_to_drop)

Para aplicar modelos como Random Forest y XGBoost, es necesario agregar características de temporalidad en los datos, para lo cuál es necesario calcular retardos, que se deben aplicar teniendo en cuenta un análisis del ACF Y PACF, así como la incorporación de los tiempos del retardo como hiperparámetros. 

Los modelos basados en árboles como Random Forest y XGBoost no son conscientes de la secuencia temporal de los datos y no pueden "extrapolar" tendencias más allá de los valores que han visto en el entrenamiento. Por lo tanto, es necesario convertir la información temporal en características que el modelo pueda entender. La creación de retardos (lags) y estadísticas de ventana móvil es la técnica estándar para lograrlo. Se podría identificar el número de retardos como un hiperparámetro, guiado por análisis de ACF y PACF (Ver EDA)

In [ ]:
# =============================================================================
# 2. INGENIERÍA DE CARACTERÍSTICAS TEMPORALES (OPTIMIZADA)
# =============================================================================



print("\nPaso 2: Realizando ingeniería de características temporales...")

TARGET = 'occurrenceCount_publisher'

def create_temporal_features_optimized(data, features_to_lag, 
                                     lags=[1, 2, 3, 4, 5], 
                                     roll_windows=[3, 5, 7],
                                     fill_na=0):
    """
    Genera características temporales y completa automáticamente con 0
    los valores NaN generados, según la lógica del negocio.
    
    Parámetros
    ----------
    data : pd.DataFrame
        Dataset con al menos 'country' y variables numéricas.
    features_to_lag : list
        Lista de columnas numéricas a transformar.
    lags : list
        Lista de retardos.
    roll_windows : list
        Lista de ventanas móviles.
    fill_na : int/float
        Valor para completar NaN (0 por defecto según lógica de negocio).
        
    Retorna
    -------
    DataFrame con nuevas características y NaN completados.
    """
    
    df_copy = data.copy()
    
    for feature in features_to_lag:
        # Características de lag
        for lag in lags:
            lag_col = f'{feature}_lag{lag}'
            df_copy[lag_col] = df_copy.groupby('country')[feature].shift(lag)
            df_copy[lag_col] = df_copy[lag_col].fillna(fill_na)
        
        # Características de ventana móvil
        for w in roll_windows:
            # Rolling mean
            mean_col = f'{feature}_rollmean{w}'
            df_copy[mean_col] = (
                df_copy.groupby('country')[feature]
                .shift(1)
                .rolling(window=w, min_periods=1)
                .mean()
            )
            df_copy[mean_col] = df_copy[mean_col].fillna(fill_na)
            
            # Rolling std
            std_col = f'{feature}_rollstd{w}'
            df_copy[std_col] = (
                df_copy.groupby('country')[feature]
                .shift(1)
                .rolling(window=w, min_periods=1)
                .std()
            )
            df_copy[std_col] = df_copy[std_col].fillna(fill_na)
    
    return df_copy

# ===========================
# Uso del código optimizado
# ===========================
features_to_lag = [
    "occurrenceCount_publisher", "pib_per_capita",
    "gasto_educacion_gobierno", "gasto_educacion_pib"
]

# Crear dataset con nuevas features (completando con 0)
df_featured = create_temporal_features_optimized(
    Data_final,
    features_to_lag=features_to_lag,
    lags=[1, 2, 3, 4, 5],  # Reducido para evitar overfitting
    roll_windows=[3, 5, 7],  # Reducido para evitar overfitting
    fill_na=0  # ¡IMPORTANTE! Completar con 0 según lógica de negocio
)

print("Ingeniería de características completada.")
print(f"Shape del dataset: {df_featured.shape}")
print(f"Valores NaN restantes: {df_featured.isnull().sum().sum()}")

# Mostrar estadísticas de las nuevas características
print("\nEstadísticas de las nuevas características:")
new_features = [col for col in df_featured.columns if any(x in col for x in ['_lag', '_roll'])]
print(df_featured[new_features].describe())



Paso 2: Realizando ingeniería de características temporales...
Ingeniería de características completada.
Shape del dataset: (656, 58)
Valores NaN restantes: 0

Estadísticas de las nuevas características:
       occurrenceCount_publisher_lag1  occurrenceCount_publisher_lag2  \
count                    6.560000e+02                    6.560000e+02   
mean                     1.530224e+07                    1.257584e+07   
std                      5.295431e+07                    4.383457e+07   
min                      0.000000e+00                    0.000000e+00   
25%                      1.549475e+04                    1.335000e+02   
50%                      1.176419e+06                    5.515200e+05   
75%                      9.340858e+06                    7.115654e+06   
max                      7.401771e+08                    5.946568e+08   

       occurrenceCount_publisher_lag3  occurrenceCount_publisher_lag4  \
count                    6.560000e+02                    6.56000

In [16]:
# Validar que no quedan NaN
assert df_featured.isnull().sum().sum() == 0, "¡Aún hay valores NaN!"

# Verificar distribución de las nuevas features
print("Distribución de valores en características temporales:")
for col in new_features:
    zero_percentage = (df_featured[col] == 0).mean() * 100
    print(f"{col}: {zero_percentage:.1f}% ceros")

Distribución de valores en características temporales:
occurrenceCount_publisher_lag1: 18.8% ceros
occurrenceCount_publisher_lag2: 25.0% ceros
occurrenceCount_publisher_lag3: 31.2% ceros
occurrenceCount_publisher_lag4: 37.5% ceros
occurrenceCount_publisher_lag5: 43.8% ceros
occurrenceCount_publisher_rollmean3: 9.9% ceros
occurrenceCount_publisher_rollstd3: 15.4% ceros
occurrenceCount_publisher_rollmean5: 5.9% ceros
occurrenceCount_publisher_rollstd5: 7.6% ceros
occurrenceCount_publisher_rollmean7: 3.7% ceros
occurrenceCount_publisher_rollstd7: 4.3% ceros
pib_per_capita_lag1: 6.2% ceros
pib_per_capita_lag2: 12.5% ceros
pib_per_capita_lag3: 18.8% ceros
pib_per_capita_lag4: 25.0% ceros
pib_per_capita_lag5: 31.2% ceros
pib_per_capita_rollmean3: 0.2% ceros
pib_per_capita_rollstd3: 0.3% ceros
pib_per_capita_rollmean5: 0.2% ceros
pib_per_capita_rollstd5: 0.3% ceros
pib_per_capita_rollmean7: 0.2% ceros
pib_per_capita_rollstd7: 0.3% ceros
gasto_educacion_gobierno_lag1: 6.2% ceros
gasto_educacio

In [21]:
# =============================================================================
# 3. PREPARACIÓN PARA EL MODELADO Y VALIDACIÓN
# =============================================================================
print("\nPaso 3: Preparando el marco de validación y los datos para el modelado...")

# Definir variable objetivo
TARGET = 'occurrenceCount_publisher'

# Definir variables predictoras (Decidir con Daniel cuáles usar basándose en EDA y disponibilidad. Preguntar si es posible usar todas sin complejizar el modelo)
features = [
    'PC1', 'PC2', 'pib_per_capita', 'gasto_educacion_gobierno',
    'gasto_educacion_pib', 'superficie_total_km2', "country", "region", "incomeLevel"
    f"{TARGET}_lag1", f"{TARGET}_lag2", f"{TARGET}_rollmean3", f"{TARGET}_rollstd3"
]

# Filtrar las variables que realmente existen en el DataFrame
features = [f for f in features if f in df_featured.columns]

# Definir X (predictoras) e y (objetivo)
X = df_featured[features].copy()
y = df_featured[TARGET].copy()

# Configurar validación cruzada para series de tiempo
n_splits = 5  # número de pliegues
tscv = TimeSeriesSplit(n_splits=n_splits)

# Diccionario para almacenar resultados
results = {
    'Prophet': None,
    'RandomForest': None,
    'XGBoost': None,
    'LSTM': None
}

print("Features seleccionadas:", features)
print("X shape:", X.shape)
print("y shape:", y.shape)



Paso 3: Preparando el marco de validación y los datos para el modelado...
Features seleccionadas: ['PC1', 'PC2', 'pib_per_capita', 'gasto_educacion_gobierno', 'gasto_educacion_pib', 'superficie_total_km2', 'country', 'region', 'occurrenceCount_publisher_lag2', 'occurrenceCount_publisher_rollmean3', 'occurrenceCount_publisher_rollstd3']
X shape: (656, 11)
y shape: (656,)


In [22]:
# =============================================================================
# 4. BUCLE DE ENTRENAMIENTO 
# =============================================================================
print("\nPaso 4: Iniciando el bucle de entrenamiento y validación de modelos...")

# Inicializar resultados correctamente
results = {
    'Prophet': [],
    'RandomForest': [],
    'XGBoost': [],
    'LSTM': []
}

unique_years = df_featured['year'].unique()
unique_years.sort()

for fold, (train_year_idx, test_year_idx) in enumerate(tscv.split(unique_years)):
    print(f"\n===== FOLD {fold + 2}/{n_splits} =====")
    
    # Identificar los años de entrenamiento y prueba
    train_years = [unique_years[i] for i in train_year_idx]
    test_years = [unique_years[i] for i in test_year_idx]
    print(f"Años de entrenamiento: {min(train_years)} - {max(train_years)}")
    print(f"Años de prueba: {min(test_years)} - {max(test_years)}")

    # Dividir los datos en entrenamiento y prueba según los años
    train_indices = df_featured[df_featured['year'].isin(train_years)].index
    test_indices = df_featured[df_featured['year'].isin(test_years)].index

    X_train, X_test = X.loc[train_indices], X.loc[test_indices]
    y_train, y_test = y.loc[train_indices], y.loc[test_indices]

    # --- Preprocesamiento DENTRO del bucle ---
    # Guardar información de países antes de procesar
    X_train_countries = X_train['country'] if 'country' in X_train.columns else None
    X_test_countries = X_test['country'] if 'country' in X_test.columns else None
    
    # Seleccionar solo características numéricas y eliminar columnas con todos NaN
    numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
    
    # Filtrar columnas que no son completamente NaN
    valid_numeric_features = []
    for col in numeric_features:
        if not X_train[col].isnull().all():  # Solo columnas con al menos algún valor
            valid_numeric_features.append(col)
    
    X_train_numeric = X_train[valid_numeric_features]
    X_test_numeric = X_test[valid_numeric_features]

    # Imputación de valores faltantes usando IterativeImputer (Puede no ser necesario ya que los datos se limpiaron y se elimnarcon columnas con NaN, incluso enla ingeniería de características)
    imputer = IterativeImputer(max_iter=10, random_state=42)
    X_train_imputed = imputer.fit_transform(X_train_numeric)
    X_test_imputed = imputer.transform(X_test_numeric)

    # Escalado de características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # Reconstruir DataFrames con las columnas válidas
    X_train_processed = pd.DataFrame(X_train_scaled, columns=valid_numeric_features, index=X_train.index)
    X_test_processed = pd.DataFrame(X_test_scaled, columns=valid_numeric_features, index=X_test.index)
    
    # Restaurar información de países si existe
    if X_train_countries is not None:
        X_train_processed['country'] = X_train_countries
    if X_test_countries is not None:
        X_test_processed['country'] = X_test_countries

    print(f"Forma de X_train_processed: {X_train_processed.shape}")
    print(f"Forma de X_test_processed: {X_test_processed.shape}")

    # CONTINUAR CON LOS MODELOS...


Paso 4: Iniciando el bucle de entrenamiento y validación de modelos...

===== FOLD 2/5 =====
Años de entrenamiento: 2007 - 2012
Años de prueba: 2013 - 2014
Forma de X_train_processed: (246, 10)
Forma de X_test_processed: (82, 10)

===== FOLD 3/5 =====
Años de entrenamiento: 2007 - 2014
Años de prueba: 2015 - 2016
Forma de X_train_processed: (328, 10)
Forma de X_test_processed: (82, 10)

===== FOLD 4/5 =====
Años de entrenamiento: 2007 - 2016
Años de prueba: 2017 - 2018
Forma de X_train_processed: (410, 10)
Forma de X_test_processed: (82, 10)

===== FOLD 5/5 =====
Años de entrenamiento: 2007 - 2018
Años de prueba: 2019 - 2020
Forma de X_train_processed: (492, 10)
Forma de X_test_processed: (82, 10)

===== FOLD 6/5 =====
Años de entrenamiento: 2007 - 2020
Años de prueba: 2021 - 2022
Forma de X_train_processed: (574, 10)
Forma de X_test_processed: (82, 10)


In [25]:
# --- Modelo 1: Prophet (Línea Base Moderna) ---
print("Entrenando Prophet...")
prophet_preds = []
y_test_prophet = []
    
regressor_cols = [col for col in features if col not in ['year', 'country'] and col in df_featured.columns]

for country in X_test_processed['country'].unique():
        train_country_df = df_featured[(df_featured['country'] == country) & (df_featured['year'].isin(train_years))].copy()
        test_country_df = df_featured[(df_featured['country'] == country) & (df_featured['year'].isin(test_years))].copy()
        
        if not train_country_df.empty and not test_country_df.empty:
            # Preparar datos para Prophet (ds, y)
            train_country_df['ds'] = pd.to_datetime(train_country_df['year'].astype(str), format='%Y')
            train_country_df.rename(columns={TARGET: 'y'}, inplace=True)
            
            # Instanciar y entrenar el modelo
            model = Prophet()
            for regressor in regressor_cols:
                if regressor in train_country_df.columns:
                    model.add_regressor(regressor)
            
            try:
                model.fit(train_country_df[['ds', 'y'] + regressor_cols])
                
                # Preparar dataframe futuro para predicción
                future_df = test_country_df.copy()
                future_df['ds'] = pd.to_datetime(future_df['year'].astype(str), format='%Y')
                
                # Predecir
                forecast = model.predict(future_df[['ds'] + regressor_cols])
                prophet_preds.extend(forecast['yhat'].values)
                y_test_prophet.extend(test_country_df[TARGET].values)
            except Exception as e:
                print(f"Error con Prophet para {country}: {e}")
                # Si el modelo falla, predecir la última observación conocida
                last_known_value = train_country_df['y'].iloc[-1] if not train_country_df.empty else 0
                prophet_preds.extend([last_known_value] * len(test_country_df))
                y_test_prophet.extend(test_country_df[TARGET].values)

if y_test_prophet and prophet_preds:
        mae = mean_absolute_error(y_test_prophet, prophet_preds)
        rmse = np.sqrt(mean_squared_error(y_test_prophet, prophet_preds))
        r2 = r2_score(y_test_prophet, prophet_preds)
        results['Prophet'].append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
        print(f"Prophet - MAE: {mae:,.2f}, RMSE: {rmse:,.2f}, R2: {r2:.2f}")

Entrenando Prophet...
Error con Prophet para Andorra: Unable to parse string "Europe & Central Asia" at position 0
Error con Prophet para Argentina: Unable to parse string "Latin America & Caribbean " at position 0
Error con Prophet para Australia: Unable to parse string "East Asia & Pacific" at position 0
Error con Prophet para Belgium: Unable to parse string "Europe & Central Asia" at position 0
Error con Prophet para Benin: Unable to parse string "Sub-Saharan Africa " at position 0
Error con Prophet para Brazil: Unable to parse string "Latin America & Caribbean " at position 0
Error con Prophet para Canada: Unable to parse string "North America" at position 0
Error con Prophet para Central African Republic: Unable to parse string "Sub-Saharan Africa " at position 0
Error con Prophet para Chile: Unable to parse string "Latin America & Caribbean " at position 0
Error con Prophet para Colombia: Unable to parse string "Latin America & Caribbean " at position 0
Error con Prophet para Cos

In [26]:
# --- Modelos de Machine Learning ---
    # Preparar datos para modelos ML (sin la columna 'country')
X_train_ml = X_train_processed.drop(columns=['country']) if 'country' in X_train_processed.columns else X_train_processed
X_test_ml = X_test_processed.drop(columns=['country']) if 'country' in X_test_processed.columns else X_test_processed
y_train_ml = y_train
y_test_ml = y_test

# --- Modelo 2: Random Forest ---
print("Entrenando Random Forest...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_ml, y_train_ml)
rf_preds = rf_model.predict(X_test_ml)
mae = mean_absolute_error(y_test_ml, rf_preds)
rmse = np.sqrt(mean_squared_error(y_test_ml, rf_preds))
r2 = r2_score(y_test_ml, rf_preds)
results['RandomForest'].append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
print(f"Random Forest - MAE: {mae:,.2f}, RMSE: {rmse:,.2f}, R2: {r2:.2f}")

Entrenando Random Forest...
Random Forest - MAE: 11,198,823.88, RMSE: 49,677,571.03, R2: 0.85


In [27]:
   # --- Modelo 3: XGBoost ---
print("Entrenando XGBoost...")
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(X_train_ml, y_train_ml)
xgb_preds = xgb_model.predict(X_test_ml)
mae = mean_absolute_error(y_test_ml, xgb_preds)
rmse = np.sqrt(mean_squared_error(y_test_ml, xgb_preds))
r2 = r2_score(y_test_ml, xgb_preds)
results['XGBoost'].append({'MAE': mae, 'RMSE': rmse, 'R2': r2})
print(f"XGBoost - MAE: {mae:,.2f}, RMSE: {rmse:,.2f}, R2: {r2:.2f}")


Entrenando XGBoost...
XGBoost - MAE: 9,453,252.70, RMSE: 40,720,390.00, R2: 0.90


In [33]:
# --- Modelo 4: LSTM ---

# --- Modelo 4: LSTM (VERSIÓN CORREGIDA) ---
print("Entrenando LSTM...")

def create_lstm_dataset_corrected(X, y, countries, look_back=3):
    """
    Versión corregida: X debe incluir los datos, countries es la serie separada
    """
    dataX, dataY = [],[]
    
    # Verificar que tenemos el mismo número de muestras en X y countries
    assert len(X) == len(countries), "X y countries deben tener la misma longitud"
    
    for country in countries.unique():
        # Obtener índices para este país
        country_indices = countries[countries == country].index
        
        # Filtrar X y y usando los índices
        X_country = X.loc[country_indices]
        y_country = y.loc[country_indices]
        
        if len(X_country) > look_back:
            for i in range(len(X_country) - look_back):
                a = X_country.iloc[i:(i + look_back)].values
                dataX.append(a)
                dataY.append(y_country.iloc[i + look_back])
    
    print(f"Secuencias creadas: {len(dataX)}")
    return np.array(dataX), np.array(dataY)

look_back = 3

# ✅ USAR X_train_ml (sin 'country') pero pasar X_train_processed['country'] separado
X_train_lstm, y_train_lstm = create_lstm_dataset_corrected(
    X_train_ml,  # Datos numéricos sin 'country'
    y_train_ml,   # Target
    X_train_processed['country'],  # Columna 'country' separada
    look_back
)

X_test_lstm, y_test_lstm = create_lstm_dataset_corrected(
    X_test_ml, 
    y_test_ml, 
    X_test_processed['country'], 
    look_back
)

print(f"Secuencias de entrenamiento: {X_train_lstm.shape}")
print(f"Secuencias de prueba: {X_test_lstm.shape}")

if X_train_lstm.shape[0] > 0 and X_test_lstm.shape[0] > 0:
    # ... resto del código LSTM
    pass  # Aquí iría el entrenamiento y predicción del modelo LSTM
else:
    print("LSTM: No hay suficientes datos para este fold")

Entrenando LSTM...
Secuencias creadas: 451
Secuencias creadas: 0
Secuencias de entrenamiento: (451, 3, 9)
Secuencias de prueba: (0,)
LSTM: No hay suficientes datos para este fold


In [ ]:
# =============================================================================
# 6. ANÁLISIS COMPARATIVO Y SELECCIÓN DEL MODELO
# =============================================================================
print("\nPaso 5: Analizando y comparando los resultados de los modelos...")

# Convertir los resultados a un DataFrame para fácil visualización
summary_list =
for model_name, metrics_list in results.items():
    if metrics_list:
        df_metrics = pd.DataFrame(metrics_list)
        summary_list.append({
            'Model': model_name,
            'MAE_mean': df_metrics['MAE'].mean(),
            'MAE_std': df_metrics['MAE'].std(),
            'RMSE_mean': df_metrics.mean(),
            'RMSE_std': df_metrics.std(),
            'R2_mean': df_metrics.mean(),
            'R2_std': df_metrics.std()
        })

summary_df = pd.DataFrame(summary_list).set_index('Model')
print("\nResumen del rendimiento de los modelos (promedio de los pliegues de CV):")
print(summary_df)

# Seleccionar el mejor modelo (basado en el RMSE más bajo)
best_model_name = summary_df.idxmin()
print(f"\nMejor modelo seleccionado: {best_model_name}")

In [ ]:
# =============================================================================
# 7. REENTRENAMIENTO DEL MODELO FINAL Y PRONÓSTICO PARA COLOMBIA
# =============================================================================
print(f"\nPaso 6: Reentrenando el modelo {best_model_name} con todos los datos...")

# Preparar todos los datos para el reentrenamiento
X_full_numeric = X.drop(columns=['country'])
imputer_final = IterativeImputer(max_iter=10, random_state=42)
X_full_imputed = imputer_final.fit_transform(X_full_numeric)
scaler_final = StandardScaler()
X_full_scaled = scaler_final.fit_transform(X_full_imputed)
X_full_processed = pd.DataFrame(X_full_scaled, columns=X_full_numeric.columns, index=X.index)

# Reentrenar el modelo XGBoost
final_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
final_model.fit(X_full_processed, y)

print("Modelo final entrenado. Generando pronósticos para Colombia...")

# --- Creación de escenarios futuros para Colombia ---
colombia_last_data = df_featured[df_featured['country'] == 'Colombia'].iloc[-1]
future_years = range(2023, 2031)
future_df = pd.DataFrame()

for year in future_years:
    future_row = colombia_last_data.copy()
    future_row['year'] = year
    future_df = pd.concat(, ignore_index=True)

# Escenario A: "Business as Usual" (extrapolación simple)
for col in:
    # Simple extrapolación lineal para la tendencia
    last_val = colombia_last_data[col]
    trend = (colombia_last_data[col] - df_featured[df_featured['country'] == 'Colombia'][col].iloc[-5:].mean()) / 5
    future_df[col] = [last_val + trend * i for i in range(1, len(future_years) + 1)]

# Escenario B: "Inversión Estratégica" (aumento del 10% en gasto en educación)
future_df_optimistic = future_df.copy()
future_df_optimistic['gasto_educacion_pib'] *= 1.10

# Función para predecir recursivamente
def generate_forecast(model, initial_data, future_template, scaler, imputer):
    history = initial_data.copy()
    predictions =
    
    for i in range(len(future_template)):
        # Preparar la fila para la predicción actual
        current_step_features = future_template.iloc[[i]]
        
        # Actualizar lags y rolling stats con el último dato conocido (de history)
        last_known = history.iloc[-1]
        for feature in features_to_lag:
            current_step_features[f'{feature}_lag_1'] = last_known[feature]
            # Para rolling stats, usamos los últimos datos de history
            rolling_window = pd.concat()])[feature]
            current_step_features[f'{feature}_rolling_mean_3'] = rolling_window.mean()
            current_step_features[f'{feature}_rolling_std_3'] = rolling_window.std()

        # Preprocesar la fila
        current_step_numeric = current_step_features[X_full_numeric.columns]
        current_step_imputed = imputer.transform(current_step_numeric)
        current_step_scaled = scaler.transform(current_step_imputed)
        
        # Predecir
        prediction = model.predict(current_step_scaled)
        predictions.append(prediction)
        
        # Actualizar 'history' con la nueva predicción para el siguiente paso
        new_row = current_step_features.copy()
        new_row = prediction
        history = pd.concat([history, new_row], ignore_index=True)
        
    return predictions

# Generar pronósticos para ambos escenarios
forecast_base = generate_forecast(final_model, df_featured[df_featured['country'] == 'Colombia'], future_df, scaler_final, imputer_final)
forecast_optimistic = generate_forecast(final_model, df_featured[df_featured['country'] == 'Colombia'], future_df_optimistic, scaler_final, imputer_final)

# --- Visualización de los resultados ---
plt.figure(figsize=(18, 9))
# Datos históricos
plt.plot(df_featured[df_featured['country'] == 'Colombia']['year'], 
         df_featured[df_featured['country'] == 'Colombia'], 
         label='Histórico - SiB Colombia', color='black', marker='o')
# Pronóstico Base
plt.plot(future_years, forecast_base, label='Pronóstico Base ("Business as Usual")', color='blue', marker='x', linestyle='--')
# Pronóstico Optimista
plt.plot(future_years, forecast_optimistic, label='Pronóstico Optimista (↑ Gasto Educación)', color='green', marker='^', linestyle='--')

plt.title('Pronóstico de Crecimiento de Publicación de Datos para SiB Colombia (2023-2030)', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Número de Registros Publicados (occurrenceCount_publisher)', fontsize=12)
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\nProceso completado.")